# Toronto Segmenting and Clustering
### Data Science Capstone Week 3 Assignment

In [1]:
# !pip install tabulate
# !pip install lxml
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

### Load the data from the wiki page into a dataframe

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df[0].head()
# print( tabulate(df[0], headers='keys', tablefmt='psql') )

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove records for Borough not assigned

In [3]:
preproc_df=df[0][df[0]['Borough'] != 'Not assigned']

### Group duplicate postcodes and concat Neighborhoods

In [4]:
preproc_df=preproc_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
preproc_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Cell has a borough but a not assigned neighborhood, then the 
### neighborhood will be the same as the borough.

In [5]:
preproc_df['Neighbourhood']=preproc_df['Neighbourhood'].apply(lambda x: preproc_df['Borough'] if 'Not assigned' in x else x)

In [6]:
preproc_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
preproc_df.shape

(103, 3)

### Create the geo_df frame from the notebook

In [10]:
%run toronto-long-lat.ipynb

In [22]:
geo_df.columns=['Postcode','Latitude','Longitude']
geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Use dataframe join on postcode to merge the frames

In [24]:
preproc_df=preproc_df.set_index('Postcode').join(geo_df.set_index('Postcode'))
preproc_df.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
